In [43]:
import pandas as pd 
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [44]:

DF_DOE = pd.read_excel('DATI_DOE_R81_ULT.xlsx')
DF_DOE


,RUN,PROVINO,Pattern,Temperature,Speed,Thickness,UTS,E,εd
0,1,2,Grid,230,10,0.1,13.813450,877.215792,0.022443
1,1,3,Grid,230,10,0.1,14.492491,913.234170,0.022406
2,1,4,Grid,230,10,0.1,12.129329,835.553666,0.019148
3,1,5,Grid,230,10,0.1,14.126136,887.405452,0.022959
4,2,1,Grid,250,50,0.2,12.374963,777.422107,0.019813
...,...,...,...,...,...,...,...,...,...
319,80,4,Lines,230,10,0.2,35.907820,1256.351635,0.024638
320,81,1,Cubic,230,50,0.4,10.254925,734.141293,0.017783
321,81,2,Cubic,230,50,0.4,10.790767,905.172547,0.016165
322,81,3,Cubic,230,50,0.4,11.066774,925.381347,0.013322


In [45]:
df = pd.get_dummies(DF_DOE, columns=['Pattern'], drop_first=False)
# Rimuovere la colonna 'Pattern_Lines' manualmente
df = df.drop(columns=['Pattern_Lines'])
df

,RUN,PROVINO,Temperature,Speed,Thickness,UTS,E,εd,Pattern_Cubic,Pattern_Grid
0,1,2,230,10,0.1,13.813450,877.215792,0.022443,0,1
1,1,3,230,10,0.1,14.492491,913.234170,0.022406,0,1
2,1,4,230,10,0.1,12.129329,835.553666,0.019148,0,1
3,1,5,230,10,0.1,14.126136,887.405452,0.022959,0,1
4,2,1,250,50,0.2,12.374963,777.422107,0.019813,0,1
...,...,...,...,...,...,...,...,...,...,...
319,80,4,230,10,0.2,35.907820,1256.351635,0.024638,0,0
320,81,1,230,50,0.4,10.254925,734.141293,0.017783,1,0
321,81,2,230,50,0.4,10.790767,905.172547,0.016165,1,0
322,81,3,230,50,0.4,11.066774,925.381347,0.013322,1,0


In [46]:
# add variable to the model 
df['Temperature_squared'] = df['Temperature'] ** 2
df['Speed_squared'] = df['Speed'] ** 2
df['Thickness_squared'] = df['Thickness'] ** 2
df['Temperature_Speed'] = df['Temperature'] * df['Speed']
df['Temperature_Thickness'] = df['Temperature'] * df['Thickness']
df['Speed_Thickness'] = df['Speed'] * df['Thickness']
df['Temperature_Pattern_Cubic'] = df['Temperature'] * df['Pattern_Cubic']
df['Temperature_Pattern_Grid'] = df['Temperature'] * df['Pattern_Grid']
df['Speed_Pattern_Cubic'] = df['Speed'] * df['Pattern_Cubic']
df['Speed_Pattern_Grid'] = df['Speed'] * df['Pattern_Grid']
df['Thickness_Pattern_Cubic'] = df['Thickness'] * df['Pattern_Cubic']
df['Thickness_Pattern_Grid'] = df['Thickness'] * df['Pattern_Grid']

In [47]:
X = df[['Temperature', 'Speed', 'Thickness', 'Pattern_Cubic', 'Pattern_Grid', 
        'Temperature_squared', 'Speed_squared', 'Thickness_squared', 'Temperature_Speed', 
        'Temperature_Thickness', 'Speed_Thickness', 'Temperature_Pattern_Cubic', 
        'Temperature_Pattern_Grid', 'Speed_Pattern_Cubic', 'Speed_Pattern_Grid', 
        'Thickness_Pattern_Cubic', 'Thickness_Pattern_Grid']]

In [48]:
X

,Temperature,Speed,Thickness,Pattern_Cubic,Pattern_Grid,Temperature_squared,Speed_squared,Thickness_squared,Temperature_Speed,Temperature_Thickness,Speed_Thickness,Temperature_Pattern_Cubic,Temperature_Pattern_Grid,Speed_Pattern_Cubic,Speed_Pattern_Grid,Thickness_Pattern_Cubic,Thickness_Pattern_Grid
0,230,10,0.1,0,1,52900,100,0.01,2300,23.0,1.0,0,230,0,10,0.0,0.1
1,230,10,0.1,0,1,52900,100,0.01,2300,23.0,1.0,0,230,0,10,0.0,0.1
2,230,10,0.1,0,1,52900,100,0.01,2300,23.0,1.0,0,230,0,10,0.0,0.1
3,230,10,0.1,0,1,52900,100,0.01,2300,23.0,1.0,0,230,0,10,0.0,0.1
4,250,50,0.2,0,1,62500,2500,0.04,12500,50.0,10.0,0,250,0,50,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,230,10,0.2,0,0,52900,100,0.04,2300,46.0,2.0,0,0,0,0,0.0,0.0
320,230,50,0.4,1,0,52900,2500,0.16,11500,92.0,20.0,230,0,50,0,0.4,0.0
321,230,50,0.4,1,0,52900,2500,0.16,11500,92.0,20.0,230,0,50,0,0.4,0.0
322,230,50,0.4,1,0,52900,2500,0.16,11500,92.0,20.0,230,0,50,0,0.4,0.0


In [49]:
X = sm.add_constant(X)



In [50]:
y = df['UTS']
model_UTS = sm.OLS(y, X)
#Fit the model
results = model_UTS.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    UTS   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     24.20
Date:                mar, 25 mar 2025   Prob (F-statistic):           8.13e-47
Time:                        13:49:33   Log-Likelihood:                -953.65
No. Observations:                 324   AIC:                             1943.
Df Residuals:                     306   BIC:                             2011.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [51]:
y = df['E']
model_E = sm.OLS(y, X)
#Fit the model
results = model_E.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      E   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.597
Method:                 Least Squares   F-statistic:                     29.20
Date:                mar, 25 mar 2025   Prob (F-statistic):           5.24e-54
Time:                        13:49:33   Log-Likelihood:                -2074.8
No. Observations:                 324   AIC:                             4186.
Df Residuals:                     306   BIC:                             4254.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [52]:
y = df['εd ']
model_ed = sm.OLS(y, X)
#Fit the model
results = model_ed.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    εd    R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     19.67
Date:                mar, 25 mar 2025   Prob (F-statistic):           1.43e-39
Time:                        13:49:33   Log-Likelihood:                 1384.2
No. Observations:                 324   AIC:                            -2732.
Df Residuals:                     306   BIC:                            -2664.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 